<a href="https://colab.research.google.com/github/TarteTatin1212/contents_search/blob/main/contents_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Senior_Project'

/content/drive/MyDrive/Senior_Project


In [ ]:
!pip install --upgrade google-api-python-client
!pip install pytz
!pip install langid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 87.8 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=d22c288ea218275bad34fe5ba11250edd0765c28c1dccfbea676d319c91c0cdd
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully bui

In [ ]:
from apiclient.discovery import build
from datetime import datetime, timedelta
import pytz
import json
import pandas as pd
import langid

In [ ]:
# APIキーをテキストファイルから読み込む
with open('/content/drive/MyDrive/Senior_Project/api_key.txt', 'r') as file:
    api_key = file.readline().strip()

# APIクライアントを構築
youtube = build("youtube", "v3", developerKey=api_key)

# 日本時間での現在の日付を取得
current_time_japan = datetime.now(pytz.timezone("Asia/Tokyo"))

# 18:00と21:00の日付オブジェクトを作成
start_time = current_time_japan.replace(hour=18, minute=0, second=0, microsecond=0)
end_time = current_time_japan.replace(hour=21, minute=0, second=0, microsecond=0)

# 日付オブジェクトをISO 8601形式の文字列に変換
start_time_iso = start_time.isoformat()
end_time_iso = end_time.isoformat()



In [ ]:
# 検索リクエストを実行
search_response = youtube.search().list(
    q="",  # 検索クエリ（ここでは全ての動画を対象）
    part="id",  # 動画のIDのみを取得
    type="video",  # 動画のみを検索対象とする
    publishedAfter=start_time_iso,  # 18:00以降に公開された動画を対象とする
    publishedBefore=end_time_iso,   # 21:00以前に公開された動画を対象とする
    maxResults=100,  # 最大100件の結果を取得
    regionCode="JP",  # 日本で公開された動画を対象とする
).execute()


# データを保存するリスト
data_japanese = []
data_other = []

# 動画ごとに必要な情報を取得
for item in search_response["items"]:
    video_id = item["id"]["videoId"]

    # 動画の詳細情報を取得
    video_response = youtube.videos().list(
        part="snippet,statistics",  # 動画の基本情報と視聴回数を取得
        id=video_id,  # 検索で取得した動画IDを指定
    ).execute()

    for item in video_response["items"]:
        title = item["snippet"]["title"]  # 動画タイトル
        lang, _ = langid.classify(title)
        channel_title = item["snippet"]["channelTitle"]  # チャンネル名
        thumbnail_url = item["snippet"]["thumbnails"]["default"]["url"]  # サムネイルURL
        view_count = item["statistics"]["viewCount"]  # 再生回数

        # 取得した情報を辞書に格納
        video_info = {
            "title": title,
            "channelTitle": channel_title,
            "thumbnailUrl": thumbnail_url,
            "viewCount": view_count
        }

        # タイトルが日本語なら data_japanese に、それ以外なら data_other に追加
        if lang == 'ja':
            data_japanese.append(video_info)
        else:
            data_other.append(video_info)


In [ ]:
# JSON形式で保存
with open('youtube_data_japanese.json', 'w') as json_file:
    json.dump(data_japanese, json_file)
with open('youtube_data_other.json', 'w') as json_file:
    json.dump(data_other, json_file)

# CSV形式で保存
df_japanese = pd.DataFrame(data_japanese)
df_japanese.to_csv('youtube_data_japanese.csv', index=False)
df_other = pd.DataFrame(data_other)
df_other.to_csv('youtube_data_other.csv', index=False)